In [1]:
#Import dependencies
import time
from selenium import webdriver
from bs4 import BeautifulSoup  as bs 

In [2]:
chromedriver = '/Users/tseringsherpa/Desktop/chromedriver'
driver = webdriver.Chrome(chromedriver)

In [3]:
def get_news():
    driver.get('https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest')
    html_string = driver.page_source
    soup = bs(html_string, 'html.parser')
    
    div = soup.find('div', attrs={'class': 'list_text'})
    try:
        title=div.findNext('div', {'class': 'content_title'}).text            
        description=div.findNext('div', {'class': 'article_teaser_body'}).text
    except:
        pass
    return {"news_title":title,"news_p":description}

In [4]:
def get_featured_image():
    driver.get('https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars')
    fullImagebtn = driver.find_element_by_id('full_image')
    fullImagebtn.click()
    time.sleep(2)

    html_string = driver.page_source
    soup = bs(html_string, 'html.parser')
    try:
        anchor = soup.find('a','ready')
        if anchor.img:
            image_url = anchor.img['src']
        featured_image_url = "https://www.jpl.nasa.gov" + image_url      
    except:
        pass
    return featured_image_url

In [5]:
def get_latest_weather():
    driver.get('https://twitter.com/marswxreport?lang=en')
    html_string = driver.page_source
    soup = bs(html_string, 'lxml')
    latest_weather = soup.find('div','js-tweet-text-container').text.strip()
    return latest_weather 

In [6]:
def get_facts():
    driver.get('https://space-facts.com/mars/')
    html_string = driver.page_source
    soup = bs(html_string, 'lxml')

    keys =[]
    values=[]
    table = soup.find('table','tablepress tablepress-id-mars')
    row_marker = 0
    for row in table.find_all('tr'):
        columns = row.find_all('td')
        keys.append(columns[0].text)
        values.append(columns[1].text)
    facts = dict(zip(keys, values))
    return facts

In [7]:
def get_individual_hemisphere(name):
    driver.get('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

    
    if name == 'Cerberus Hemisphere Enhanced':
        getFullImagebtn = driver.find_element_by_xpath("//*[text()='Cerberus Hemisphere Enhanced']")
    elif name == 'Schiaparelli Hemisphere Enhanced':
         getFullImagebtn = driver.find_element_by_xpath("//*[text()='Schiaparelli Hemisphere Enhanced']")
    elif name == 'Syrtis Major Hemisphere Enhanced':
         getFullImagebtn = driver.find_element_by_xpath("//*[text()='Syrtis Major Hemisphere Enhanced']")
    elif name == 'Valles Marineris Hemisphere Enhanced':
         getFullImagebtn = driver.find_element_by_xpath("//*[text()='Valles Marineris Hemisphere Enhanced']")
    else:
        pass
    
    getFullImagebtn.click()

    html_string = driver.page_source
    soup = bs(html_string, 'lxml')

    image_url="https://astrogeology.usgs.gov" + str (soup.find('img','wide-image')['src'])
    return({"title": name, "img_url": image_url})

def get_hemispheres():
    hemisphere_image_urls=[]
    h1 = get_individual_hemisphere('Cerberus Hemisphere Enhanced')
    hemisphere_image_urls.append(h1)
    h2 = get_individual_hemisphere('Schiaparelli Hemisphere Enhanced')
    hemisphere_image_urls.append(h2)
    h3 = get_individual_hemisphere('Syrtis Major Hemisphere Enhanced')
    hemisphere_image_urls.append(h3)
    h4 = get_individual_hemisphere('Valles Marineris Hemisphere Enhanced')
    hemisphere_image_urls.append(h4)
    return hemisphere_image_urls

In [8]:
def scrape():
    output ={}
    news=get_news()
    featured_image_url= get_featured_image()
    latest_weather=get_latest_weather()
    facts =get_facts()
    hemisphere_image_urls =get_hemispheres()
    output ={ "news":news,"featured_image_url":featured_image_url,"weather":latest_weather,"facts":facts, "hemisphere_image_urls":hemisphere_image_urls
    }
    return output  

In [9]:
output = scrape()
import json
print(json.dumps(output,indent=4))

{
    "news": {
        "news_title": "NASA's First Deep-Space CubeSats Say: 'Polo!'",
        "news_p": "MarCO is a pair of tiny spacecraft that launched with NASA's InSight lander today."
    },
    "featured_image_url": "https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19046_ip.jpg",
    "weather": "Sol 2043 (May 06, 2018), Sunny, high -14C/6F, low -71C/-95F, pressure at 7.36 hPa, daylight 05:22-17:20",
    "facts": {
        "Equatorial Diameter:": "6,792 km\n",
        "Polar Diameter:": "6,752 km\n",
        "Mass:": "6.42 x 10^23 kg (10.7% Earth)",
        "Moons:": "2 (Phobos & Deimos)",
        "Orbit Distance:": "227,943,824 km (1.52 AU)",
        "Orbit Period:": "687 days (1.9 years)\n",
        "Surface Temperature: ": "-153 to 20 \u00b0C",
        "First Record:": "2nd millennium BC",
        "Recorded By:": "Egyptian astronomers"
    },
    "hemisphere_image_urls": [
        {
            "title": "Cerberus Hemisphere Enhanced",
            "img_url": "https://a